In [58]:
acme_data = """MBI|FNAME|LNAME|DOB|EMAIL|PHONE
1234567890A|John|Doe|03/15/1955|JOHN.DOE@EMAIL.COM|5551234567
9876543210B|Jane|Smith|07/22/1948|jane.smith@email.com|5559876543
"""

with open("/content/acme.txt", "w") as f:
    f.write(acme_data)


In [59]:
bettercare_data = """subscriber_id,first_name,last_name,date_of_birth,email,phone
BC-001,Alice,Johnson,1965-08-10,alice.j@test.com,555-222-3333
BC-002,Charlie,Brown,1972-03-25,charlie.b@test.com,5554445555
"""

with open("/content/bettercare.csv", "w") as f:
    f.write(bettercare_data)


In [60]:
PARTNER_CONFIG = {
    "acme": {
        "file_path": "/content/acme.txt",
        "delimiter": "|",
        "partner_code": "ACME",
        "column_mapping": {
            "MBI": "external_id",
            "FNAME": "first_name",
            "LNAME": "last_name",
            "DOB": "dob",
            "EMAIL": "email",
            "PHONE": "phone"
        }
    },
    "bettercare": {
        "file_path": "/content/bettercare.csv",
        "delimiter": ",",
        "partner_code": "BETTERCARE",
        "column_mapping": {
            "subscriber_id": "external_id",
            "first_name": "first_name",
            "last_name": "last_name",
            "date_of_birth": "dob",
            "email": "email",
            "phone": "phone"
        }
    }
}


In [61]:
def format_dob(dob):
    if pd.isna(dob):
        return None

    date_formats = ["%m/%d/%Y", "%Y-%m-%d"]

    for fmt in date_formats:
        try:
            return datetime.strptime(str(dob), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass

    return None


In [62]:
def format_phone(phone):
    if pd.isna(phone):
        return None

    cleaned = re.sub(r"\D", "", str(phone))

    if re.fullmatch(r"\d{10}", cleaned):
        return "-".join([cleaned[:3], cleaned[3:6], cleaned[6:]])

    return None


In [63]:
def validate_records(df):
    invalid_mask = (
        df["external_id"].isna() |
        df["dob"].isna() |
        df["phone"].isna()
    )

    bad_records = df[invalid_mask]
    good_records = df[~invalid_mask]

    return good_records, bad_records


In [64]:
def ingest_partner(partner_name, config):
    cfg = config.get(partner_name)

    try:
        df = pd.read_csv(
            cfg["file_path"],
            delimiter=cfg["delimiter"]
        )
    except Exception as err:
        print(f"Failed to read file for {partner_name}: {err}")
        return pd.DataFrame()


    df.rename(columns=cfg["column_mapping"], inplace=True)
    df = df[cfg["column_mapping"].values()]


    df["first_name"] = df["first_name"].str.title()
    df["last_name"] = df["last_name"].str.title()
    df["email"] = df["email"].str.lower()
    df["dob"] = df["dob"].map(format_dob)
    df["phone"] = df["phone"].map(format_phone)
    df["partner_code"] = cfg["partner_code"]


    valid_df, invalid_df = validate_records(df)

    print(
        f"{partner_name.upper()} | "
        f"Total: {len(df)} | "
        f"Valid: {len(valid_df)} | "
        f"Rejected: {len(invalid_df)}"
    )

    return valid_df


In [65]:
all_dataframes = []

for partner_name in PARTNER_CONFIG.keys():
    partner_df = ingest_partner(partner_name, PARTNER_CONFIG)
    all_dataframes.append(partner_df)

final_df = pd.concat(all_dataframes, ignore_index=True)


ACME | Total: 2 | Valid: 2 | Rejected: 0
BETTERCARE | Total: 2 | Valid: 2 | Rejected: 0


In [66]:
final_df

,external_id,first_name,last_name,dob,email,phone,partner_code
0,1234567890A,John,Doe,1955-03-15,john.doe@email.com,555-123-4567,ACME
1,9876543210B,Jane,Smith,1948-07-22,jane.smith@email.com,555-987-6543,ACME
2,BC-001,Alice,Johnson,1965-08-10,alice.j@test.com,555-222-3333,BETTERCARE
3,BC-002,Charlie,Brown,1972-03-25,charlie.b@test.com,555-444-5555,BETTERCARE


In [67]:
with open("/content/final_eligibility_output.csv", "w", newline="") as f:
    final_df.to_csv(f, index=False)

In [68]:
def validate_records(df):
    invalid_mask = (
        df["external_id"].isna() |
        df["dob"].isna() |
        df["phone"].isna()
    )

    bad_records = df[invalid_mask]
    good_records = df[~invalid_mask]

    return good_records, bad_records


In [69]:
summary = final_df.groupby("partner_code").agg(
    total_records=("external_id", "count"),
    unique_members=("external_id", "nunique")
)

summary


,total_records,unique_members
partner_code,,
ACME,2,2
BETTERCARE,2,2


In [70]:
excel_path = "/content/final_eligibility_output.xlsx"
final_df.to_excel(excel_path, index=False)

print("Excel file saved at:", excel_path)


Excel file saved at: /content/final_eligibility_output.xlsx
